In [73]:
import winreg, sys, os
from winreg import *

In [76]:
class RegIterer:
    def __init__(self):
        pass
    
    def onValue(self, name, data, typ, path):
        pass  # hook
    
    def onKey(self, key, path):
        self.iterKeys(key, path)
        self.iterValues(key, path)
    
    def iterValues(self, key, path):
        i = 0
        while True:
            try:
                name, data, typ = EnumValue(key, i)
            except OSError:
                break
            self.onValue(name, data, typ, path + "\\" + name)
            i += 1
            
    def iterKeys(self, key, path=""):
        i = 0
        while True:
            try:
                name = EnumKey(key, i)
            except OSError:
                break
            try:
                subkey = OpenKey(key, name, 0, KEY_READ | KEY_QUERY_VALUE)
            except PermissionError:
                pass
                # print("Unable to access key: ", path+"\\"+name)
            else:
                self.onKey(subkey, path+"\\"+name)
                # no CloseKey - handled by python finalizer
            i += 1

            
class ExtSearcher(RegIterer):
    def __init__(self, filename, exts):
        self.filename = filename
        self.file = None
        self.exts = tuple(set(exts))
        
        self.startwith_filtered = {
            "C:\\Users\\Administrator\\AppData\\Local\\Microsoft\\Windows\\Notifications\\wpnidm",
            "wpnidm:https://site-cdn.onenote.net/",
            "@{Microsoft."
        }
        self.keys_filtered = {
            "Software\\Classes\\Extensions\\ContractId",
            "Software\\Classes\\Local Settings\\MrtCache\\"
        }
        
        
    def onValue(self, name, data, typ, path):
        p2 = path.split("\\",1)[1]
        for kf in self.keys_filtered:
            if p2.startswith(kf):
                return
        if typ == REG_SZ:
            if any(e in data for e in self.exts):
                if any(data.startswith(f) for f in self.startwith_filtered):
                    return
                dname = os.path.basename(data)
                print("%-50s :: %s"%(dname, path), file=(self.file or sys.stdout))
                
    def doiterKeys(self, key, path):
        with open(self.filename, 'w') as f:
            self.file = f
            self.iterKeys(key, path)

In [63]:
fn = "C:\\users\\administrator\\documents\\reg.txt"
searcher = ExtSearcher(fn, [".jpg",".avi",".rar",".zip",".png"])

searcher.doiterKeys(HKEY_CURRENT_USER, "HKEY_CURRENT_USER")

In [68]:
class XSearcher(RegIterer):
    def onValue(self, name, data, typ, path):
        if typ == REG_SZ:
            if data.startswith("X:"):
                print("%-50s :: %s"%(path, data))

In [69]:
hkeys = [
    "HKEY_CLASSES_ROOT", 
    "HKEY_CURRENT_USER",
    "HKEY_LOCAL_MACHINE",
    "HKEY_USERS",
    "HKEY_CURRENT_CONFIG"
]

for hkey in hkeys:
    XSearcher().iterKeys(globals()[hkey], hkey)

HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Uninstall\{3196C62F-9C7B-4392-88B4-05C037D05518}_is1\Inno Setup: App Path :: X:\Freenet
HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Uninstall\{3196C62F-9C7B-4392-88B4-05C037D05518}_is1\InstallLocation :: X:\Freenet\
HKEY_USERS\S-1-5-21-3480973611-4170425562-2601081845-500\Software\Microsoft\Windows\CurrentVersion\Uninstall\{3196C62F-9C7B-4392-88B4-05C037D05518}_is1\Inno Setup: App Path :: X:\Freenet
HKEY_USERS\S-1-5-21-3480973611-4170425562-2601081845-500\Software\Microsoft\Windows\CurrentVersion\Uninstall\{3196C62F-9C7B-4392-88B4-05C037D05518}_is1\InstallLocation :: X:\Freenet\


In [78]:
hkeys = [
    "HKEY_CLASSES_ROOT", 
    "HKEY_CURRENT_USER",
    "HKEY_LOCAL_MACHINE",
    "HKEY_USERS",
    "HKEY_CURRENT_CONFIG"
]

for hkey in hkeys:
    fn = "C:\\users\\administrator\\documents\\reg_%s.txt"%hkey
    searcher = ExtSearcher(fn, [".gif",".jpeg"])
    searcher.doiterKeys(globals()[hkey], hkey)